In [ ]:
# Import Deps
import pandas as pd
import numpy as np
import tensorflow as tf
import logging
from tqdm import tqdm

# Import Locals
from src.utils.consts import TF_RECORD_DATASET, TF_BUFFER_SIZE
from src.model.tensorflow_utils import load_and_split_dataset
from src.model.tensorflow_utils import setup_logger, setup_training_logger, setup_weight_monitor


# Input Data
DATASET_SIZE = 102697
tfrecord_path = f"{TF_RECORD_DATASET}/chest_xray_data.tfrecord"

In [ ]:
# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

In [ ]:
# Load Dataset
batch_size = 32
shuffle_buffer_size = 1000
tfrecord_buffer_size = TF_BUFFER_SIZE
dataset_size = DATASET_SIZE

train_ds, val_ds, test_ds = load_and_split_dataset(
    tfrecord_path, 
    batch_size, 
    shuffle_buffer_size, 
    tfrecord_buffer_size, 
    dataset_size
)

In [ ]:
# DensNet121 like Model
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, MaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Concatenate, AveragePooling2D
from tensorflow.keras.models import Model

def conv_block(x, filters, kernel_size=(3,3), strides=1, padding="same"):
    """
    Podstawowa warstwa konwolucyjna z BatchNorm i ReLU
    """
    x = Conv2D(
        filters, 
        kernel_size, s
        trides=strides, 
        padding=padding, 
        use_bias=False,
        # L2 Regularization
        kernel_regularizer=l2(1e-4),
        kernel_initializer='he_normal'
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x

def dense_block(x, num_layers, growth_rate):
    """
    Dense Block - łączy wyjścia wielu warstw konwolucyjnych
    """
    concat_features = [x]
    for _ in range(num_layers):
        conv_out = conv_block(x, growth_rate)
        concat_features.append(conv_out)
        x = Concatenate()(concat_features)  # Połączenie cech z poprzednich warstw
    return x

def transition_layer(x, compression=0.5):
    """
    Redukuje liczbę cech po każdym bloku
    """
    filters = int(tf.keras.backend.int_shape(x)[-1] * compression)  # Zmniejsza liczbę kanałów
    x = conv_block(x, filters, kernel_size=(1,1))  # Redukcja wymiarów przez 1x1 Conv
    x = AveragePooling2D(pool_size=(2,2), strides=2, padding="same")(x)  # Pooling
    return x

def build_densenet121(num_classes: int) -> Model:
    """
    Implementacja DenseNet121 od podstaw.
    """
    inputs = Input(shape=(224, 224, 3), name="input_layer")

    # Warstwa Początkowa
    x = conv_block(inputs, filters=64, kernel_size=(7,7), strides=2, padding="same")
    x = MaxPooling2D(pool_size=(3,3), strides=2, padding="same")(x)

    # Dense Block 1
    x = dense_block(x, num_layers=6, growth_rate=32)
    # Transition Layer 1
    x = transition_layer(x)                          

    # Dense Block 2
    x = dense_block(x, num_layers=12, growth_rate=32)
    # Transition Layer 2
    x = transition_layer(x)                           

    # Dense Block 3
    x = dense_block(x, num_layers=24, growth_rate=32)
    # Transition Layer 3
    x = transition_layer(x)                         

    # Dense Block 4
    x = dense_block(x, num_layers=16, growth_rate=32)

    # Global Average Pooling
    x = GlobalAveragePooling2D()(x)

    # Dropout, żeby uniknąć przeuczenia
    x = Dropout(0.5)(x)

    # Warstwa wyjściowa (multi-label classification)
    outputs = Dense(num_classes, activation="sigmoid", name="output_layer")(x)

    # Tworzenie modelu
    model = Model(inputs=inputs, outputs=outputs, name="CustomDenseNet121")

    return model

# Tworzenie modelu dla 14 klas
num_classes = 14
model = build_densenet121(num_classes)

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

# Kompilacja modelu
model.compile(optimizer=Adam(learning_rate=0.0001), loss=BinaryCrossentropy(), metrics=["accuracy"])

# Checkpoint - zapisuje najlepszy model
checkpoint = ModelCheckpoint("best_model.h5", monitor="val_loss", save_best_only=True, mode="min")

# ReduceLROnPlateau - zmniejsza LR, gdy walidacja przestaje się poprawiać
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6, verbose=1)

# Trenowanie modelu
train_ds = train_ds.take(1000)
val_ds = val_ds.take(200)
test_ds = test_ds.take(100)
steps_per_epoch = len(list(train_ds))  # Count number of batches in dataset
logger = setup_logger()
training_logger = setup_training_logger(logger, 100)
weight_monitor = setup_weight_monitor(logger)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,
    steps_per_epoch=steps_per_epoch,
    callbacks=[checkpoint, reduce_lr, training_logger, weight_monitor]
)